In [11]:
from torch_geometric.datasets import Planetoid
import torch
from torch_geometric.data import Data

In [12]:
dataset_cora = Planetoid(root='./cora/', name='Cora')
# dataset = Planetoid(root='./citeseer',name='Citeseer')
# dataset = Planetoid(root='./pubmed/',name='Pubmed')
print(dataset_cora[0])

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [1]:
import transformers
import torch

/home/btr/miniconda3/envs/LLMEnG/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from transformers import AutoTokenizer, AutoModel
import torch

# 定义模型和tokenizer的本地路径
local_model_path = "/home/btr/bpmn/model/safetensors/Meta-Llama-3-8B-Instruct"

# 加载预训练的tokenizer和模型
tokenizer = AutoTokenizer.from_pretrained(local_model_path)
model = AutoModel.from_pretrained(local_model_path)

# 要处理的输入文本
text = "这是一个示例文本"

# 将文本转换为tokenizer的输入格式
inputs = tokenizer(text, return_tensors="pt")

# 获取模型的最后一层隐藏状态作为嵌入表示
with torch.no_grad():
    outputs = model(**inputs)

# 获取最后一层隐藏状态
last_hidden_states = outputs.last_hidden_state

# 输出嵌入表示
print(last_hidden_states)      

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.47s/it]


tensor([[[ 4.1851, -0.2059, -1.8382,  ..., -2.8908,  1.3605,  0.3109],
         [-1.8622, -0.9529, -2.4024,  ...,  1.0467,  1.3186,  1.0996],
         [-0.2761, -0.4898, -0.9106,  ...,  1.2816,  0.6039,  1.7683],
         ...,
         [-0.3205, -2.7684, -2.1332,  ...,  1.1992,  0.9619,  1.0644],
         [ 2.1264, -4.0235, -3.8833,  ..., -1.4924,  1.3204,  1.5962],
         [-2.7754, -2.8158, -1.3051,  ..., -1.8278, -0.0244,  1.6202]]])


In [7]:
last_hidden_states.shape

torch.Size([1, 7, 4096])

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv
from torch_geometric.nn import GATConv

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = SAGEConv(dataset_cora.num_node_features, 16, 'lstm')
        self.conv2 = SAGEConv(16, dataset_cora.num_classes, 'lstm')

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.softmax(x, dim=1)

        return x
    
class GAT(nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
        self.conv1 = GATConv(dataset_cora.num_node_features, 16, heads=2)
        self.conv2 = GATConv(2*16, dataset_cora.num_classes, heads=1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.softmax(x, dim=1)

        return x

In [23]:
model = GAT()
print(model)

GAT(
  (conv1): GATConv(1433, 16, heads=2)
  (conv2): GATConv(32, 7, heads=1)
)


In [24]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device)
data = dataset_cora[0].to(device)
print(data)

cuda:1
Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [25]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [26]:
model.train()
for epoch in range(200):
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    _, pred = torch.max(out[data.train_mask], dim=1)
    correct = (pred == data.y[data.train_mask]).sum().item()
    acc = correct/data.train_mask.sum().item()

    print('Epoch {:03d} train_loss: {:.4f} train_acc: {:.4f}'.format(
        epoch, loss.item(), acc))

Epoch 000 train_loss: 1.9451 train_acc: 0.1857
Epoch 001 train_loss: 1.9166 train_acc: 0.7214
Epoch 002 train_loss: 1.8723 train_acc: 0.8071
Epoch 003 train_loss: 1.8173 train_acc: 0.7786
Epoch 004 train_loss: 1.7426 train_acc: 0.8000
Epoch 005 train_loss: 1.6530 train_acc: 0.8929
Epoch 006 train_loss: 1.5969 train_acc: 0.8714
Epoch 007 train_loss: 1.5263 train_acc: 0.8571
Epoch 008 train_loss: 1.4645 train_acc: 0.9143
Epoch 009 train_loss: 1.4076 train_acc: 0.9500
Epoch 010 train_loss: 1.3652 train_acc: 0.9429
Epoch 011 train_loss: 1.3142 train_acc: 0.9714
Epoch 012 train_loss: 1.2917 train_acc: 0.9571
Epoch 013 train_loss: 1.2634 train_acc: 0.9714
Epoch 014 train_loss: 1.2558 train_acc: 0.9714
Epoch 015 train_loss: 1.2349 train_acc: 0.9857
Epoch 016 train_loss: 1.2239 train_acc: 0.9786
Epoch 017 train_loss: 1.2058 train_acc: 0.9857
Epoch 018 train_loss: 1.2125 train_acc: 0.9714
Epoch 019 train_loss: 1.2058 train_acc: 0.9786
Epoch 020 train_loss: 1.1977 train_acc: 0.9786
Epoch 021 tra

: 